In [1]:
import warnings
import logging
warnings.filterwarnings("ignore", module="presidio_analyzer")
logging.getLogger("presidio-analyzer").setLevel(logging.ERROR)

In [2]:
!pip install --upgrade --quiet langchain langchain-openai langchain-experimental presidio-analyzer presidio-anonymizer spacy Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 63.

In [3]:
from langchain_experimental.data_anonymizer import PresidioAnonymizer
anonymizer = PresidioAnonymizer()
anonymizer.anonymize(
    "My name is Denny Gosh, call me at 324-455-5634 or email me at danny.sh@gmail.com"
)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


'My name is Jeffrey Espinoza PhD, call me at 615-606-1968x196 or email me at qterrell@example.com'

In [4]:
from langchain_experimental.data_anonymizer import PresidioAnonymizer
anonymizer = PresidioAnonymizer()
anonymizer.anonymize(
    "My name is Denny Gosh, call me at 324-455-5634 and my repo number is 12434 and my email is real@Task.com"
)

'My name is Jocelyn Koch, call me at 001-879-287-9536x00260 and my repo number is 1998-01-11 and my email is danielstrickland@example.org'

In [5]:
from langchain_experimental.data_anonymizer import PresidioAnonymizer
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON"])
anonymizer.anonymize(
    "My name is Denny Gosh, call me at 324-455-5634 or email me at danny.sh@gmail.com"
)

'My name is Omar Rogers, call me at 324-455-5634 or email me at danny.sh@gmail.com'

In [6]:
from faker import Faker

fake = Faker(locale="en_US")

def fake_number(_: str = None) -> str:
    return str(fake.random.randint(1000, 5000))

from presidio_anonymizer.entities import OperatorConfig
from presidio_analyzer import PatternRecognizer, Pattern
from langchain_experimental.data_anonymizer import PresidioAnonymizer

anonymizer = PresidioAnonymizer()

repo_number_pattern = Pattern(
    name="repo_number_pattern",
    regex=r"(?<=\D)\d{5}(?=\D)",
    score=1
)

repo_recognizer = PatternRecognizer(
    supported_entity="REPO_NUMBER",
    patterns=[repo_number_pattern]
)

anonymizer.add_recognizer(repo_recognizer)

new_operator = {
    "REPO_NUMBER": OperatorConfig(
        "custom", {"lambda": fake_number}
    )
}

anonymizer.add_operators(new_operator)

anonymized_text = anonymizer.anonymize(
    "My name is Denny Gosh, call me at 324-455-5634 and my repo number is 12434 and my email is real@Task.com"
)

print(anonymized_text)


My name is Jared King, call me at 864-665-9453x620 and my repo number is 2848 and my email is vangjoshua@example.com


In [7]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer = PresidioReversibleAnonymizer(
    add_default_faker_operators=False
)
anonymized_text = anonymizer.anonymize(
    "My name is Denny Gosh, call me at 324-455-5634 and my repo number is 12434 and my email is real@Task.com"
)
print(anonymized_text)

My name is <PERSON>, call me at <PHONE_NUMBER> and my repo number is <DATE_TIME> and my email is <EMAIL_ADDRESS>
